In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
import sklearn

In [2]:
data = pd.read_csv("data.csv")
display(data)

,天气,上课距离,学生成绩,课程类别,上课情况
0,晴,远,差,选修,逃课
1,晴,近,差,必修,上课
2,晴,近,好,必修,上课
3,阴,远,差,选修,逃课
4,阴,近,好,选修,上课
5,阴,近,好,必修,上课
6,雨,远,差,选修,逃课
7,雨,近,好,必修,上课
8,雨,近,差,必修,逃课
9,雨,远,好,选修,逃课


In [3]:
X, y = data.iloc[:10, :-1], data.iloc[:10, -1]
oe = OrdinalEncoder()
X = oe.fit_transform(X)
print(X)
# 输出每个特征的类别信息。
print(oe.categories_)

[[0. 1. 1. 1.]
 [0. 0. 1. 0.]
 [0. 0. 0. 0.]
 [1. 1. 1. 1.]
 [1. 0. 0. 1.]
 [1. 0. 0. 0.]
 [2. 1. 1. 1.]
 [2. 0. 0. 0.]
 [2. 0. 1. 0.]
 [2. 1. 0. 1.]]
[array(['晴', '阴', '雨'], dtype=object), array(['近', '远'], dtype=object), array(['好', '差'], dtype=object), array(['必修', '选修'], dtype=object)]


In [4]:
le = LabelEncoder()
y = le.fit_transform(y)
print(y)
# 输出标签的类别信息。
print(le.classes_)

[1 0 0 1 0 0 1 0 1 1]
['上课' '逃课']


In [5]:
from sklearn.naive_bayes import CategoricalNB
cnb = CategoricalNB()
cnb.fit(X, y)
# 每个类别中，每个特征各个取值出现的次数。
# 该值为列表类型，列表中元素形状依次为(类别数量, 对应特征的类别取值数量）。
print(cnb.category_count_)
# 每个类别的样本数量。
print(cnb.class_count_)
# 每个类别的对数概率，如果想查看原始概率，需要使用指数还原。
print(np.exp(cnb.class_log_prior_))
# 类别的标签值。
print(cnb.classes_)
# 计算P(x_i|y)的概率。
print([np.exp(item) for item in cnb.feature_log_prob_])
# 特征的数量。
print(cnb.n_features_)

[array([[2., 2., 1.],
       [1., 1., 3.]]), array([[5., 0.],
       [1., 4.]]), array([[4., 1.],
       [1., 4.]]), array([[4., 1.],
       [1., 4.]])]
[5. 5.]
[0.5 0.5]
[0 1]
[array([[0.375, 0.375, 0.25 ],
       [0.25 , 0.25 , 0.5  ]]), array([[0.85714286, 0.14285714],
       [0.28571429, 0.71428571]]), array([[0.71428571, 0.28571429],
       [0.28571429, 0.71428571]]), array([[0.71428571, 0.28571429],
       [0.28571429, 0.71428571]])]
4


In [6]:
# 对最后两条记录进行预测。注意，测试集的数据需要与训练集
# 执行相同的转换。
test = data.iloc[10:, :-1]
test = oe.transform(test)
y_hat = cnb.predict(test)
print(y_hat)
pro = cnb.predict_proba(test)
print(pro)

[1 1]
[[0.41860465 0.58139535]
 [0.23076923 0.76923077]]


In [7]:
a = 0.008
b = 0.0128
c = a + b
print(a / c, b / c)

0.38461538461538464 0.6153846153846154


In [8]:
# alpha：指定平滑系数，默认为1（拉普拉斯平滑）。
cnb = CategoricalNB(alpha=0)
cnb.fit(X, y)
pro = cnb.predict_proba(test)
print(pro)

[[3.84615385e-01 6.15384615e-01]
 [5.00000000e-11 1.00000000e+00]]


D:\software\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:507: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


In [9]:
from sklearn.naive_bayes import GaussianNB
np.random.seed(0)
X = np.random.randint(0, 10, size=(6, 2))
y = np.array([0, 0, 0, 1, 1, 1])
data = pd.DataFrame(np.concatenate([X, y.reshape(-1, 1)], axis=1), columns=["x1", "x2", "y"])
display(data)
gnb = GaussianNB()
gnb.fit(X, y)
# 每个类别的先验概率。P(y)
print("概率：", gnb.class_prior_)
# 每个类别样本的数量。
print("样本数量：", gnb.class_count_)
# 每个类别的标签。
print("标签", gnb.classes_)
# 每个特征在每个类别下的均值。
print("均值：", gnb.theta_)
# 每个特征在每个类别下的方差。
print("方差：", gnb.sigma_)
# 测试集
X_test = np.array([[6, 3]])
print("预测结果：", gnb.predict(X_test))
print("预测结果概率：", gnb.predict_proba(X_test))

,x1,x2,y
0,5,0,0
1,3,3,0
2,7,9,0
3,3,5,1
4,2,4,1
5,7,6,1


概率： [0.5 0.5]
样本数量： [3. 3.]
标签 [0 1]
均值： [[5. 4.]
 [4. 5.]]
方差： [[ 2.66666667 14.00000001]
 [ 4.66666667  0.66666667]]
预测结果： [0]
预测结果概率： [[0.87684687 0.12315313]]


In [10]:
from sklearn.naive_bayes import BernoulliNB
np.random.seed(0)
X = np.random.randint(-5, 5, size=(6, 2))
y = np.array([0, 0, 0, 1, 1, 1])
data = pd.DataFrame(np.concatenate([X, y.reshape(-1, 1)], axis=1), columns=["x1", "x2", "y"])
display(data)
bnb = BernoulliNB()
bnb.fit(X, y)
# 每个特征在每个类别下发生（出现）的次数。因为伯努利分布只有两个值，
# 我们只需要计算出现的概率P(x=1|y)，不出现的概率P(x=0|y)使用1减去P(x=1|y)即可。
print("数值1出现次数：", bnb.feature_count_)
# 每个类别样本所占的比重，即P(y)。注意，该值为概率取对数之后的结果，如果需要查看
# 原有的概率，需要使用指数还原。
print("类别占比p(y)：", np.exp(bnb.class_log_prior_))
# 每个类别下，每个特征（值为1）所占的比例（概率），即P(x|y)。注意，该值为概率
# 取对数之后的结果，如果需要查看原有的概率，需要使用指数还原。
print("特征概率：", np.exp(bnb.feature_log_prob_))

,x1,x2,y
0,0,-5,0
1,-2,-2,0
2,2,4,0
3,-2,0,1
4,-3,-1,1
5,2,1,1


数值1出现次数： [[1. 1.]
 [1. 1.]]
类别占比p(y)： [0.5 0.5]
特征概率： [[0.4 0.4]
 [0.4 0.4]]


In [11]:
from sklearn.naive_bayes import MultinomialNB
np.random.seed(0)
X = np.random.randint(0, 4, size=(6, 2))
y = np.array([0, 0, 0, 1, 1, 1])
data = pd.DataFrame(np.concatenate([X, y.reshape(-1, 1)], axis=1), columns=["x1",
"x2", "y"])
display(data)
mnb = MultinomialNB()
mnb.fit(X, y)
# 每个类别的样本数量。
print(mnb.class_count_)
# 每个特征在每个类别下发生（出现）的次数。
print(mnb.feature_count_)
# 每个类别下，每个特征所占的比例（概率），即P(x|y)。注意，该值为概率
# 取对数之后的结果，如果需要查看原有的概率，需要使用指数还原。
print(np.exp(mnb.feature_log_prob_))

,x1,x2,y
0,0,3,0
1,1,0,0
2,3,3,0
3,3,3,1
4,1,3,1
5,1,2,1


[3. 3.]
[[4. 6.]
 [5. 8.]]
[[0.41666667 0.58333333]
 [0.4        0.6       ]]


In [12]:
from sklearn.naive_bayes import ComplementNB
np.random.seed(0)
X = np.random.randint(0, 4, size=(6, 2))
y = np.array([0, 0, 0, 1, 1, 1])
data = pd.DataFrame(np.concatenate([X, y.reshape(-1, 1)], axis=1), columns=["x1",
"x2", "y"])
display(data)
cnb = ComplementNB()
cnb.fit(X, y)
# 每个类别的样本数量。
print(cnb.class_count_)
# 每个特征在每个类别下发生（出现）的次数。
print(cnb.feature_count_)
# 特征i不在指定类别中发生的概率（对数概率的相反数）。
print(cnb.feature_log_prob_)

,x1,x2,y
0,0,3,0
1,1,0,0
2,3,3,0
3,3,3,1
4,1,3,1
5,1,2,1


[3. 3.]
[[4. 6.]
 [5. 8.]]
[[0.91629073 0.51082562]
 [0.87546874 0.5389965 ]]


In [13]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,
random_state=0)
models = [("分类朴素贝叶斯", CategoricalNB()),
("高斯朴素贝叶斯：", GaussianNB()),
("伯努利朴素贝叶斯：", BernoulliNB()),
("多项式朴素贝叶斯：", MultinomialNB()),
("互补朴素贝叶斯：", ComplementNB())
]
for name, m in models:
    m.fit(X_train, y_train)
    print(name, m.score(X_test, y_test))

分类朴素贝叶斯 0.8947368421052632
高斯朴素贝叶斯： 1.0
伯努利朴素贝叶斯： 0.23684210526315788
多项式朴素贝叶斯： 0.5789473684210527
互补朴素贝叶斯： 0.5789473684210527
